<a href="https://colab.research.google.com/github/JacksonXi/optimisation/blob/main/import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
import sys
import os.path
import numpy as np
import pandas as pd

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))
    
from pyomo.environ import *


     |████████████████████████████████| 8.9 MB 4.6 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

In [2]:
!git clone https://github.com/JacksonXi/optimisation

Cloning into 'optimisation'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 49 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [3]:
cd /content/optimisation/import

/content/optimisation/import


**Generated energy < non-controlable energy**




In [4]:
Flexibleindex = pd.read_csv("flexibleIndex.csv")
Flexibleindex

,Unnamed: 0,TV,Heater,WashingMachine,Dryer,Fridger,Airconditioner,InductionCooker
0,C1,0,0,1,1,0,1,0
1,C2,1,0,1,0,0,1,0
2,C3,1,0,0,1,0,1,0
3,C4,0,1,0,1,1,0,1
4,C5,1,0,1,0,1,0,0
5,C6,0,1,0,1,0,1,0
6,C7,0,0,0,1,0,1,1
7,C8,1,0,0,1,0,0,1
8,C9,0,0,0,1,0,0,0
9,C10,1,1,1,0,0,0,1


**Obtain consumer ID**

In [5]:
Consumer = list(Flexibleindex['Unnamed: 0'])
Consumer1 = list(Flexibleindex['Unnamed: 0'])
Consumer

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']

In [6]:
flexibleindex = pd.read_csv("flexibleIndex.csv", index_col=0)
print(flexibleindex)

     TV  Heater  WashingMachine  ...  Fridger  Airconditioner  InductionCooker
C1    0       0               1  ...        0               1                0
C2    1       0               1  ...        0               1                0
C3    1       0               0  ...        0               1                0
C4    0       1               0  ...        1               0                1
C5    1       0               1  ...        1               0                0
C6    0       1               0  ...        0               1                0
C7    0       0               0  ...        0               1                1
C8    1       0               0  ...        0               0                1
C9    0       0               0  ...        0               0                0
C10   1       1               1  ...        0               0                1

[10 rows x 7 columns]


**Obtain flexibleindex during peak hours**

**0: ON**

**1: OFF**

In [7]:
power = pd.read_csv('power.csv', index_col=0)
power

,TV,Heater,WashingMachine,Dryer,Fridger,Airconditioner,InductionCooker
C1,2,4,4,8,2,5,5
C2,2,7,4,5,1,3,5
C3,2,5,2,5,2,4,4
C4,2,8,3,5,1,4,4
C5,2,5,4,5,2,3,6
C6,1,6,2,6,1,5,5
C7,1,7,4,5,1,5,6
C8,1,7,2,6,1,3,4
C9,1,4,4,6,1,5,5
C10,2,8,2,5,1,4,5


In [8]:
bid = pd.read_csv('bid.csv', index_col=0)
bid

,bid
C1,18
C2,15
C3,19
C4,21
C5,22
C6,20
C7,17
C8,16
C9,18
C10,21


In [9]:
demand = pd.read_csv('demand.csv', index_col=0)
demand

,demand
C1,3
C2,13
C3,1
C4,1
C5,5
C6,3
C7,4
C8,6
C9,8
C10,5


**Sorting the bid**

In [10]:
n = len(bid)
consumer = [] 
while n != 0:
    max = Consumer[0]
    for i in range(0,n):
      if bid['bid'][max] < bid['bid'][Consumer[i]]:
          max = Consumer[i]
      else:
          max = max
    consumer.append(max)      
    Consumer.remove(max)
    n -= 1
print(Consumer)
print(consumer)

[]
['C5', 'C4', 'C10', 'C6', 'C3', 'C1', 'C9', 'C7', 'C8', 'C2']


In [11]:
generatedEnergy = pd.read_csv('generated energy.csv')
Prosumer = list(generatedEnergy['Unnamed: 0'])
Prosumer1 = list(generatedEnergy['Unnamed: 0'])
Prosumer

['P1', 'P2', 'P3', 'P4', 'P5', 'P6']

**Sorting the storage energy**

In [12]:
generated_energy = pd.read_csv('generated energy.csv', index_col=0)
generated_energy

,generated energy
P1,6
P2,10
P3,4
P4,3
P5,2
P6,2


In [13]:
n = len(generated_energy)
prosumer = [] 
while n != 0:
    max = Prosumer[0]
    for i in range(0,n):
      if generated_energy['generated energy'][max] < generated_energy['generated energy'][Prosumer[i]]:
          max = Prosumer[i]
      else:
          max = max
    prosumer.append(max)      
    Prosumer.remove(max)
    n -= 1
print(prosumer)
print(Prosumer)

['P2', 'P1', 'P3', 'P4', 'P5', 'P6']
[]


In [14]:
appliances = list(flexibleindex.keys())
appliances

['TV',
 'Heater',
 'WashingMachine',
 'Dryer',
 'Fridger',
 'Airconditioner',
 'InductionCooker']

**Energy is allocated in the order of the bid amount**

In [15]:
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

model.transport = Var(prosumer, consumer , domain = NonNegativeReals)

model.revenue = Objective(expr = sum(model.transport[p, c]*bid['bid'][c] for p in prosumer for c in consumer), 
                           sense = maximize)

model.supply = ConstraintList()
for p in prosumer:
      model.supply.add(sum(model.transport[p, c] for c in consumer) <= generated_energy['generated energy'][p])

model.demand = ConstraintList()
for c in consumer:
      model.demand.add(sum(model.transport[p, c] for p in prosumer) <= demand['demand'][c])

results = SolverFactory('cbc').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 530.0
  Upper bound: 530.0
  Number of objectives: 1
  Number of constraints: 17
  Number of variables: 61
  Number of nonzeros: 60
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created 

In [16]:
model.pprint()

5 Set Declarations
    demand_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    supply_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    transport_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                              : Size : Members
        None :     2 : transport_index_0*transport_index_1 :   60 : {('P2', 'C5'), ('P2', 'C4'), ('P2', 'C10'), ('P2', 'C6'), ('P2', 'C3'), ('P2', 'C1'), ('P2', 'C9'), ('P2', 'C7'), ('P2', 'C8'), ('P2', 'C2'), ('P1', 'C5'), ('P1', 'C4'), ('P1', 'C10'), ('P1', 'C6'), ('P1', 'C3'), ('P1', 'C1'), ('P1', 'C9'), ('P1', 'C7'), ('P1', 'C8'), ('P1', 'C2'), ('P3', 'C5'), ('P3', 'C4'), ('P3', 'C10'), ('P3', 'C6'), ('P3', 'C3'), ('P3', 'C1'), ('P3', 'C9'), ('P3', 'C7'), ('P3', 'C8'), ('P3', 'C2'), ('P4', 'C5'), ('P4', 

In [17]:
for p in prosumer:
    for c in consumer:
        print(p, c, model.transport[p,c]())

P2 C5 3.0
P2 C4 1.0
P2 C10 4.0
P2 C6 0.0
P2 C3 0.0
P2 C1 0.0
P2 C9 2.0
P2 C7 0.0
P2 C8 0.0
P2 C2 0.0
P1 C5 0.0
P1 C4 0.0
P1 C10 0.0
P1 C6 0.0
P1 C3 0.0
P1 C1 0.0
P1 C9 6.0
P1 C7 0.0
P1 C8 0.0
P1 C2 0.0
P3 C5 0.0
P3 C4 0.0
P3 C10 0.0
P3 C6 0.0
P3 C3 1.0
P3 C1 3.0
P3 C9 0.0
P3 C7 0.0
P3 C8 0.0
P3 C2 0.0
P4 C5 0.0
P4 C4 0.0
P4 C10 1.0
P4 C6 2.0
P4 C3 0.0
P4 C1 0.0
P4 C9 0.0
P4 C7 0.0
P4 C8 0.0
P4 C2 0.0
P5 C5 0.0
P5 C4 0.0
P5 C10 0.0
P5 C6 1.0
P5 C3 0.0
P5 C1 0.0
P5 C9 0.0
P5 C7 1.0
P5 C8 0.0
P5 C2 0.0
P6 C5 2.0
P6 C4 0.0
P6 C10 0.0
P6 C6 0.0
P6 C3 0.0
P6 C1 0.0
P6 C9 0.0
P6 C7 0.0
P6 C8 0.0
P6 C2 0.0


In [23]:
if 'ok' == str(results.Solver.status):
    print("Total Community Revenue = ",model.revenue())
    print("\nTransport Table:")
    for p in Prosumer1:
        for c in Consumer1:
            if model.transport[p,c]() > 0:
                print("Transport from ", p," to ", c, ":", model.transport[p,c]())
else:
    print("No Valid Solution Found")

Total Community Revenue =  530.0

Transport Table:
Transport from  P1  to  C9 : 6.0
Transport from  P2  to  C4 : 1.0
Transport from  P2  to  C5 : 3.0
Transport from  P2  to  C9 : 2.0
Transport from  P2  to  C10 : 4.0
Transport from  P3  to  C1 : 3.0
Transport from  P3  to  C3 : 1.0
Transport from  P4  to  C6 : 2.0
Transport from  P4  to  C10 : 1.0
Transport from  P5  to  C6 : 1.0
Transport from  P5  to  C7 : 1.0
Transport from  P6  to  C5 : 2.0


In [19]:
n = 0
if 'ok' == str(results.Solver.status):
    print("Personal Revenue:\n")
    for p in prosumer:
        for c in consumer:
            if model.transport[p,c]() > 0:
                print("Balance from ", c," to ", p, ":", model.transport[p,c]() * bid['bid'][c])
else:
    print("No Valid Solution Found")

Personal Revenue:

Balance from  C5  to  P2 : 66.0
Balance from  C4  to  P2 : 21.0
Balance from  C10  to  P2 : 84.0
Balance from  C9  to  P2 : 36.0
Balance from  C9  to  P1 : 108.0
Balance from  C3  to  P3 : 19.0
Balance from  C1  to  P3 : 54.0
Balance from  C10  to  P4 : 21.0
Balance from  C6  to  P4 : 40.0
Balance from  C6  to  P5 : 20.0
Balance from  C7  to  P5 : 17.0
Balance from  C5  to  P6 : 44.0


In [20]:
n = 0
if 'ok' == str(results.Solver.status):
    print("Personal Revenue:\n")
    for p in prosumer:
      for c in consumer:
          n += model.transport[p, c]() * bid['bid'][c] 
      print("The revenue of",p ,":" , n)
      n = 0
else:
    print("No Valid Solution Found")

Personal Revenue:

The revenue of P2 : 207.0
The revenue of P1 : 108.0
The revenue of P3 : 73.0
The revenue of P4 : 61.0
The revenue of P5 : 37.0
The revenue of P6 : 44.0


In [21]:
n = 0
if 'ok' == str(results.Solver.status):
    print("The imported electricity form the grid:\n")
    for c in consumer:
        n = demand['demand'][c]
        for p in prosumer:
            n -= model.transport[p, c]()
        if n > 0 :
            print(c, "needs to import ",n , "KWh") 

The imported electricity form the grid:

C7 needs to import  3.0 KWh
C8 needs to import  6.0 KWh
C2 needs to import  13.0 KWh


In [22]:
n = 0
if 'ok' == str(results.Solver.status):
    print("Personal Revenue:\n")
    for c in Consumer1:
      for p in Prosumer1:
          n += model.transport[p, c]()
      print("The received energy of",c ,":" , n)
      n = 0
else:
    print("No Valid Solution Found")

Personal Revenue:

The received energy of C1 : 3.0
The received energy of C2 : 0.0
The received energy of C3 : 1.0
The received energy of C4 : 1.0
The received energy of C5 : 5.0
The received energy of C6 : 3.0
The received energy of C7 : 1.0
The received energy of C8 : 0.0
The received energy of C9 : 8.0
The received energy of C10 : 5.0
